# why GANs

**Generative model:** For the purposes of this tutorial, "any model that takes a training set, consisting of samples drawn from a distribution $p_{data}$, and learns to represent an estimate of that distribution somehow." [Goodfellow 2016]

### demo: first-order Markov model

A simple model many might be familiar with that can be seen as generative is a **first-order Markov model**, 
which models transitions between states as a matrix.

Say that our data a vector fo values indicating whether it rained or not for the last 25 days (where a value of `1` means it rained).

In [19]:
import numpy as np

rained = np.random.randint(2, size=25)
print(rained)

[0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0]


How would we "fit" our Markov model?

What we want is a matrix where each element represents the probability of transition from state $i$ to state $j$.

To compute this we count the occurrence of each possible transition `{0 -> 0, 0 ->1, 1 -> 0, 1 -> 1}` and then normalize the counts in each row by the sum of counts in that row.

In [47]:
def fit_markov(data):
    trans_mat = np.zeros((2, 2))

    for ind, transition in enumerate(rained):
        if ind == 0:
            prev_state = transition
        else:
            trans_mat[prev_state][transition] += 1
            prev_state = transition

    trans_mat /= trans_mat.sum(1)[:, np.newaxis]
    
    return trans_mat

In [49]:
trans_mat = fit_markov(rained)
print(trans_mat)

[[0.61538462 0.38461538]
 [0.45454545 0.54545455]]


Now we can *generate* new data from the distribution we have fit.

A simple way to do so is, for each time step:
- take the row of our matrix corresponding to our current state
- sampling from the uniform distribution
- use that sample to decide which state we are in

In [51]:
def generate(n_steps, trans_mat):
    current_state = np.random.randint(2, size=1).item()  # initial state, assume 50 / 50 chance of rain or no for example
    fake = [current_state]

    for t in range(n_steps):
        trans_mat_this_state = np.cumsum(trans_mat[current_state])
        current_state = np.where(
             trans_mat_this_state > np.random.uniform()
        )[0][0]
        fake.append(current_state)

    fake = np.asarray(fake)
    return fake

In [52]:
n_steps = 25

fake_rained = generate(n_steps, trans_mat)

print(fake_rained)

[1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0]


## a taxonomy of generative models
explicit v. implicit

## why do we want generative models in our research?

1. test "our ability to represent and manipulate high-dimensional probability distributions" [Goodfellow 2016]

### demo: GalaxyGAN

2. "many tasks intrinsically require realistic generation of samples from some distribution." [Goodfellow 2016]

### demo: single-image super resolution

3. fit models that would otherwise be intractable or computationally expensive
  + e.g., require variational and/or Monte Carlo approximation